##  Работа 8.1
## Определение постоянных Стефана-Больцмана и Планка из анализа теплового излучения накаленного тела.
#### Карпов Алексей
<br> **Цель работы:** При помощи модели абсолютно черного тела (АЧТ) проводятся измерения температуры оптическим пирометром с исчезающей нитью и термопарой, исследуется излучение накаленных тел с различной испускательной способностью, определяются постоянные Стефана-Больцмана и Планка.

### Теоретическая часть 
<br>
Закон Стефана-Больцмана для АЧТ:
    $$W = \sigma S (T^4 - T_0^4) $$
<br> Для серого тела, с учетом большой разницы между температурой самого тела и
комнатной температуры:
    $$W=\varepsilon_T \sigma ST^4,$$
 <br> где Стефана-Больцмана определятся как: $$\sigma = \frac{2\pi^5k^4_Б}{15c^2h^3}$$
 <br> Сотвественно постоянная планка: $$h =\sqrt[3]{ \frac{2\pi^5k^4_Б}{15c^2\sigma}}$$
<br> 
<img src="./img1.png"/>
<img src="./img2.png"/>

In [2]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize

In [57]:
def linear(coeffs, x):
    return coeffs[0] * x + coeffs[1]
def loss(coeffs):
    return sum((linear(coeffs, x_i) - y_i)**2 for x_i, y_i in zip(x, y))

### Ход Работы.
<br>
1. Определим температуру АЧТ при помощи пирометра при повышении и понижении температуры проволоки пирометра: 
 $$T_p =  1005^\circ C$$
2. Определим температуру АЧТ при помощи термопары, учитывая, что постоянная термопары $40 мквВ/^\circ C$: 
 $$T_t = 996 \pm 12 ^\circ C$$
3. Проверим закон Стефана Больцмана:


In [174]:
pd.options.display.float_format = '{:,.3f}'.format
table = pd.read_excel("table.xlsx")
kwargs = {"σW": lambda x: (0.01 / x.v + 0.01/x.I) * x.W}
table = table.assign(**kwargs)
kwargs = {"$T_t$" : lambda x: 120.0/122 * x["T_p"] + 2800.0/122}
table = table.assign(**kwargs)
kwargs = {"σ$T_t$" : lambda x: 5 / x["T_p"] * x["$T_t$"]}
table = table.assign(**kwargs)
kwargs = {"$\ln{T_t}$" : lambda x: np.log(x["$T_t$"]), "σ$\ln{T_t}$" : lambda x: x["σ$T_t$"] / x["$T_t$"] * x["$\ln{T_t}$"]}
table = table.assign(**kwargs)
kwargs = {"$\ln{W}$" : lambda x: np.log(x["W"]), "σ$\ln{W}$" : lambda x: x["σW"] / x["W"] * x["$\ln{W}$"]}
table = table.assign(**kwargs)
table.rename(index=str, columns={"I":"I, A", "v":"V, B", "T_p":"$T_p, ^\circ K$", "W":"W, Вт", "$T_t$":"$T_t,^\circ K$"},inplace=True)
table.rename(index=str, columns={"σW": "σW, Вт", "σ$T_t$":"σ$T_t, ^\circ K$"},inplace=True)

In [175]:
table

,"I, A","V, B","$T_p, ^\circ K$","W, Вт","σW, Вт","$T_t,^\circ K$","σ$T_t, ^\circ K$",$\ln{T_t}$,σ$\ln{T_t}$,$\ln{W}$,σ$\ln{W}$
0,0.440,1.120,1163,0.496,0.016,"1,166.885",5.017,7.062,0.030,-0.701,-0.022
1,0.489,1.850,1297,0.905,0.023,"1,298.689",5.007,7.169,0.028,-0.100,-0.003
2,0.563,2.520,1466,1.419,0.031,"1,464.918",4.996,7.290,0.025,0.350,0.008
3,0.600,2.880,1537,1.728,0.035,"1,534.754",4.993,7.336,0.024,0.547,0.011
4,0.650,3.490,1635,2.293,0.042,"1,631.148",4.988,7.397,0.023,0.830,0.015
5,0.668,4.170,1708,2.798,0.049,"1,702.951",4.985,7.440,0.022,1.029,0.018
6,0.778,4.830,1791,3.758,0.056,"1,784.590",4.982,7.487,0.021,1.324,0.020
7,0.850,5.730,1894,4.870,0.066,"1,885.902",4.979,7.542,0.020,1.583,0.021
8,0.941,6.950,1979,6.540,0.079,"1,969.508",4.976,7.586,0.019,1.878,0.023
9,1.006,7.900,1800,7.947,0.089,"1,793.443",4.982,7.492,0.021,2.073,0.023


In [1]:
x = table["$\ln{T_t}$"]
y = table["$\ln{W}$"]
dx = table["σ$\ln{T_t}$"]
dy = table["σ$\ln{W}$"]
coefs = minimize(loss, [1,1]).x

plt.rcParams['figure.figsize'] = 8, 8
plt.rcParams['axes.grid'] = True
pd.set_option('display.max_columns', None)
plt.rc('text', usetex=False)
plt.rc('font', family='serif')
plt.rc('font', weight='bold')
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10)
plt.xlabel("$\ln{T_t}$")
plt.ylabel("$\ln{W}$")
plt.errorbar(x, y, xerr=dx, yerr=dy, fmt='x', color='w', ecolor='red', barsabove=True)
plt.plot(x, [linear(coefs, x_i) for x_i in x], 'b-')
print("y = %f * x - %f" %(coefs[0], abs(coefs[1])))

NameError: name 'table' is not defined

In [162]:
dn = (np.mean(dx / x) + np.mean(dy / y)) + np.sqrt(sum([(y_i - linear(coefs, x_i))**2 for x_i,y_i in zip(x,y)]))/np.size(y) 
print("n = %.2f" %coefs[0])
print("σn = %.2f" %(dn * coefs[0]))

n = 4.81
σn = 0.19


<br>
<br>
<br>
<br>
<br>

4. Определим величину постоянной Стефана-Больцмана и Планка для всех температур больше 1700 $^\circ K$:
<br> 
Ожидаем получить: 
$$\sigma = 5,670367 \cdot 10^{-8}$$
$$ h = 6,626 \cdot 10^{−34}$$

In [176]:
table = pd.read_excel("table2.xlsx")
table

,T,σ,σσ,h,σh
0,1702,6.510,0.420,4.740,0.306
1,1784,6.560,0.340,4.722,0.245
2,1885,6.190,0.320,4.842,0.250
3,1969,5.880,0.290,5.095,0.251
4,2061,5.830,0.280,5.384,0.259
